# Imports

In [1]:
import os, re, math
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer
import unicodedata
from IPython.display import display, HTML
from sklearn.metrics import classification_report, confusion_matrix

C:\Users\semde\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Kan opgegeven procedure niet vinden'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Config

In [2]:
# === Config: RobBERT v2 + neutral band en gewichten (titel/lead/body) ===
ROB_BERT_SENTIMENT = "DTAI-KULeuven/robbert-v2-dutch-sentiment"

DATA_DIR = "data"
OUT_CSV  = "out/sentiment_results.csv"

from dataclasses import dataclass

@dataclass
class Weights:
    title: float = 1.0
    lead:  float = 1.0
    body:  float = 1.0   # body weer meewegen

WEIGHTS = Weights()

# Neutral band voor beslissing op basis van signed score (p_pos - p_neg)
NEUTRAL_BAND = 0

# Chunk-instellingen (gebruik je al elders)
MAX_TOKENS = 400
STRIDE     = 50

# Load Title, Lead and Body Texts

In [3]:
# === Nieuw: inlezen uit Title_Lead_Body.xlsx ===
# Pad aanpassen indien nodig; nu staat hij naast het notebook.
XLSX_PATH = "out/Title_Lead_Body.xlsx"

df_input = pd.read_excel(XLSX_PATH)

# Zorg dat de kolommen precies 'id', 'title', 'lead', 'body' heten
expected = {"id","title","lead","body"}
missing = expected - set(df_input.columns.str.lower())
if missing:
    raise ValueError(f"Ontbrekende kolommen in {XLSX_PATH}: {missing}. "
                     "Zorg voor kolommen: id, title, lead, body.")

# Normaliseer kolomnamen en vul lege waarden op
df_input = df_input.rename(columns={c: c.lower() for c in df_input.columns})
df_input[["title","lead","body"]] = df_input[["title","lead","body"]].fillna("")
print(df_input.head())
print(f"Aantal rijen ingelezen: {len(df_input)}")

   id                                              title  \
0   7  Nederlandse patiënt wacht te lang op betere me...   
1  10  Nieuwe kankermedicijnen leveren meer financiël...   
2  11         Hoe controleer je verstopte moedervlekken?   
3  16  'Ik vind het erg als 'n infuus van 25.000 euro...   
4  21  Wachtlijsten en personeelstekort: het 'zorginf...   

                                                lead  \
0  Wat een prachtig bericht onlangs, dat meer kan...   
1  Vorige week verscheen in Trouw een artikel met...   
2  Meer dan twintig jaar geleden ontdekte ze op h...   
3  Waarom schrijven artsen 1005 milligram van een...   
4  De gezondheidszorg is 'op', er zit geen rek me...   

                                                body              bron  
0  Maar het is jammer dat het zo lang duurt voord...             Trouw  
1  Nederland is een mooi land waarin uiteindelijk...     de Volkskrant  
2  Eerst over die dagelijkse inspectie. Dat is ec...             Trouw  
3  Ziekenh

# Load Model

In [4]:
# === Alleen de RobBERT v2 sentiment pipeline laden ===
ROB_NAME = "DTAI-KULeuven/robbert-v2-dutch-sentiment"

def load_pipe(name, tries=2):
    last = None
    for i in range(tries):
        try:
            clf = pipeline(
                task="sentiment-analysis",
                model=name,
                tokenizer=name,
                top_k=None,          # return_all_scores vervangen
                truncation=True
            )
            return clf
        except Exception as e:
            last = e
    raise last

rob_pipe = load_pipe(ROB_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(ROB_NAME)

Device set to use cpu


# Helpers

In [29]:
def make_chunks_by_tokens(text: str, tokenizer, max_tokens: int = 400, stride: int = 50) -> List[str]:
    if not text.strip():
        return []
    toks = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    i = 0
    while i < len(toks):
        window = toks[i:i+max_tokens]
        if not window:
            break
        chunk = tokenizer.decode(window, skip_special_tokens=True)
        chunks.append(chunk)
        if i + max_tokens >= len(toks):
            break
        i += max_tokens - stride
    return chunks

def chunk_body(text: str, tokenizer, prefer_paragraphs: bool = True, max_tokens: int = 300, stride: int = 50) -> List[str]:
    if not text.strip():
        return []
    if prefer_paragraphs:
        paras = re.split(r"\\n\\s*\\n", text.strip())
        paras = [p.strip() for p in paras if p.strip()]
        chunks = []
        for p in paras:
            if len(tokenizer.encode(p, add_special_tokens=False)) <= max_tokens:
                chunks.append(p)
            else:
                chunks.extend(make_chunks_by_tokens(p, tokenizer, max_tokens=max_tokens, stride=stride))
        return chunks
    else:
        return make_chunks_by_tokens(text, tokenizer, max_tokens=max_tokens, stride=stride)


LABEL_MAP = {
    "POSITIVE": "positief", "NEGATIVE": "negatief", "NEUTRAL": "neutraal",
    "Positive": "positief", "Negative": "negatief", "Neutral": "neutraal",
    "positief": "positief", "negatief": "negatief", "neutraal": "neutraal"
}

def normalize_pnn(probs: dict) -> dict:
    import numpy as np
    arr = np.array([
        probs.get("positief", 0.0),
        probs.get("negatief", 0.0),
        probs.get("neutraal", 0.0)
    ], dtype=float)
    s = arr.sum()
    if s <= 0:
        return {"positief":0.0, "negatief":0.0, "neutraal":1.0}
    arr = arr / s
    return {"positief": float(arr[0]), "negatief": float(arr[1]), "neutraal": float(arr[2])}

def score_text_with_pipe(text: str, clf) -> dict:
    text = (text or "").strip()
    if not text:
        return {"positief":0.0, "negatief":0.0, "neutraal":1.0}
    out = clf(text, truncation=True)
    scores = out[0]  # top_k=None -> lijst van dicts
    probs = {"positief":0.0, "negatief":0.0, "neutraal":0.0}
    for item in scores:
        lab = LABEL_MAP.get(item["label"])
        if lab:
            probs[lab] = float(item["score"])
    return normalize_pnn(probs)

MAX_MODEL_TOKENS = 400
HEADROOM = 8
SAFE_MAX = MAX_MODEL_TOKENS - HEADROOM

def token_chunks(text, tokenizer, max_tokens=SAFE_MAX, stride=50):
    if not text or not text.strip():
        return []
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) <= max_tokens:
        return [text]
    out, i = [], 0
    while i < len(ids):
        j = min(i + max_tokens, len(ids))
        chunk_ids = ids[i:j]
        out.append(tokenizer.decode(chunk_ids, clean_up_tokenization_spaces=True))
        if j >= len(ids): break
        i = max(j - stride, i + 1)
    return out

def aggregate_article_with_pipe_binary(title, lead, body_chunks, clf, tokenizer,
                                       weights=None):
    """
    Alleen POS/NEG. Negeert neutraal volledig.
    - Titel/lead/body worden gechunked zoals voorheen.
    - Weeg t/l/b via weights (default title=2.0, lead=1.0, body=1.0).
    - Label = 'positief' als p_pos >= p_neg, anders 'negatief'.
    """
    if weights is None:
        from dataclasses import dataclass
        @dataclass
        class W: title: float = 1.0; lead: float = 1.0; body: float = 1.0
        weights = W()

    parts = []

    # Titel
    t_chunks = token_chunks(title or "", tokenizer, max_tokens=SAFE_MAX, stride=50)
    w_t_each = (weights.title / max(len(t_chunks), 1)) if t_chunks else 0.0
    for t in t_chunks:
        if t.strip():
            parts.append((t.strip(), w_t_each))

    # Lead
    l_chunks = token_chunks(lead or "", tokenizer, max_tokens=SAFE_MAX, stride=50)
    w_l_each = (weights.lead / max(len(l_chunks), 1)) if l_chunks else 0.0
    for l in l_chunks:
        if l.strip():
            parts.append((l.strip(), w_l_each))

    # Body (al gechunked upstream)
    if body_chunks:
        w_b_each = weights.body / len(body_chunks)
        for ch in body_chunks:
            ch = (ch or "").strip()
            if ch:
                parts.append((ch, w_b_each))

    if not parts:
        # lege tekst -> kies 'negatief' conservatief of geef pos=neg=0.5
        return {"p_pos":0.5, "p_neg":0.5, "label":"negatief"}

    acc_pos = 0.0
    acc_neg = 0.0
    total_w = 0.0

    for txt, w in parts:
        out = clf(txt, truncation=True, max_length=MAX_MODEL_TOKENS, padding=False)
        scores = out[0]  # lijst met dicts
        p_pos = p_neg = 0.0
        for item in scores:
            lab = LABEL_MAP.get(item["label"])
            if lab == "positief":
                p_pos = float(item["score"])
            elif lab == "negatief":
                p_neg = float(item["score"])
            # 'neutraal' negeren we bewust

        # Her-normaliseer over POS/NEG alleen (optioneel, maar aan te raden)
        s = p_pos + p_neg
        if s > 0:
            p_pos2 = p_pos / s
            p_neg2 = p_neg / s
        else:
            # als model iets geks geeft, maak gelijk verdeeld
            p_pos2 = p_neg2 = 0.5

        acc_pos += p_pos2 * w
        acc_neg += p_neg2 * w
        total_w += w

    if total_w <= 0:
        total_w = 1.0
    p_pos_bin = acc_pos / total_w
    p_neg_bin = acc_neg / total_w

    label = "positief" if p_pos_bin >= p_neg_bin else "negatief"
    return {"p_pos": float(p_pos_bin), "p_neg": float(p_neg_bin), "label": label}

# NLP Classification Analysis

In [30]:
rows = []
for _, r in tqdm(df_input.iterrows(), total=len(df_input)):
    art_id = r["id"]
    title  = r["title"] or ""
    lead   = r["lead"]  or ""
    body   = r["body"]  or ""

    # Body chunken zoals voorheen
    body_chunks = chunk_body(
        body,
        TOKENIZER,
        prefer_paragraphs=True,
        max_tokens=MAX_TOKENS,
        stride=STRIDE
    )

    # Binaire aggregatie (past bij je bestaande pipeline)
    rob_bin = aggregate_article_with_pipe_binary(
        title, lead, body_chunks,
        rob_pipe, TOKENIZER
    )

    rows.append({
        "id": art_id,
        "title": title,
        "lead": lead,
        "rob_p_pos": rob_bin["p_pos"],
        "rob_p_neg": rob_bin["p_neg"],
        "rob_label": rob_bin["label"],  # 'positief' of 'negatief'
    })

df = pd.DataFrame(rows)
print(df.head())
print(f"Klaar. {len(df)} rijen geclassificeerd.")

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [01:59<00:00,  1.70s/it]

   id                                              title  \
0   7  Nederlandse patiënt wacht te lang op betere me...   
1  10  Nieuwe kankermedicijnen leveren meer financiël...   
2  11         Hoe controleer je verstopte moedervlekken?   
3  16  'Ik vind het erg als 'n infuus van 25.000 euro...   
4  21  Wachtlijsten en personeelstekort: het 'zorginf...   

                                                lead  rob_p_pos  rob_p_neg  \
0  Wat een prachtig bericht onlangs, dat meer kan...   0.411551   0.588449   
1  Vorige week verscheen in Trouw een artikel met...   0.391569   0.608431   
2  Meer dan twintig jaar geleden ontdekte ze op h...   0.693425   0.306575   
3  Waarom schrijven artsen 1005 milligram van een...   0.338410   0.661590   
4  De gezondheidszorg is 'op', er zit geen rek me...   0.494780   0.505220   

  rob_label  
0  negatief  
1  negatief  
2  positief  
3  negatief  
4  negatief  
Klaar. 70 rijen geclassificeerd.


In [31]:
df.to_csv("out/sentiment_results.csv", index=False, encoding="utf-8")
print("[DONE] Geschreven naar out/sentiment_results.csv")

[DONE] Geschreven naar out/sentiment_results.csv


# Classification Results

In [32]:
# === Load results (RobBERT only) ===

# 1) Resultaten ophalen
#    - bij voorkeur de in-memory 'df' uit je classificatieloop
#    - anders uit de CSV (legacy)
df_results = pd.read_csv("out/sentiment_results.csv")

# 2) Zorg voor een nette 'id' kolom (int)
def to_int_id(v):
    try:
        return int(v)
    except Exception:
        return pd.NA

if 'id' not in df_results.columns:
    if 'file' in df_results.columns:
        import re
        def extract_id_legacy(val):
            s = str(val)
            m = re.search(r'bericht[_\s-]*(\d+)', s, flags=re.I)
            if m: return int(m.group(1))
            m2 = re.search(r'(\d+)', s)
            return int(m2.group(1)) if m2 else pd.NA
        df_results['id'] = df_results['file'].apply(extract_id_legacy)
    else:
        df_results['id'] = np.arange(1, len(df_results) + 1)

df_results['id'] = df_results['id'].apply(to_int_id)

# 3) rob_label normaliseren
if 'rob_label' not in df_results.columns:
    raise KeyError("Column 'rob_label' ontbreekt in de resultaten.")
df_results['rob_label'] = (
    df_results['rob_label'].astype(str).str.strip().str.lower()
)

df_labels = df_results[['id', 'rob_label']].copy()

# 4) One-hot encoding met alleen aanwezige klassen
present_classes = sorted(df_labels['rob_label'].dropna().unique().tolist())
df_encoded = pd.get_dummies(df_labels, columns=['rob_label'], prefix='robbert')

print(df_encoded.head())

   id  robbert_negatief  robbert_positief
0   7              True             False
1  10              True             False
2  11             False              True
3  16              True             False
4  21              True             False


In [33]:
print(df_encoded.tail())

     id  robbert_negatief  robbert_positief
65  271              True             False
66  287             False              True
67  296             False              True
68  300             False              True
69  306             False              True


In [34]:
# 5) Samenvatting (positief/negatief/neutraal als aanwezig)
summary = {}
for cls in ['positief', 'negatief', 'neutraal']:
    col = f'robbert_{cls}'
    if col in df_encoded.columns:
        summary[cls] = int(df_encoded[col].sum())

summary_df = pd.DataFrame([summary], index=['robbert']).fillna(0).astype(int)
summary_df.index.name = "Model"
print(summary_df)


         positief  negatief
Model                      
robbert        51        19


# Analyse tov human sentiment (maaike)

In [35]:
# === Evaluatie: Human vs RobBERT (binaire evaluatie: positief vs negatief) ===

# 1) Resultaten ophalen (in-memory 'df' heeft de voorkeur)
try:
    df_results = df.copy()
except NameError:
    df_results = pd.read_csv("out/sentiment_results.csv")

# 2) Human labels inladen
df_h = pd.read_excel("out/Human_Sentiment.xlsx")  # verwacht: Artikel, Sentiment

# 3) IDs netjes naar int (of None)
def to_int_or_none(x):
    try:
        return int(x)
    except Exception:
        return None

if 'id' not in df_results.columns:
    raise KeyError("Resultaten missen een 'id' kolom.")
df_results['id'] = df_results['id'].apply(to_int_or_none)

if not pd.api.types.is_integer_dtype(df_h['Artikel']):
    df_h['Artikel'] = df_h['Artikel'].apply(to_int_or_none)

# 4) Labels normaliseren
df_h['Human_Label'] = df_h['Sentiment'].astype(str).str.strip().str.lower()
# alleen binaire klassen
df_h = df_h[df_h['Human_Label'].isin({'positief','negatief'})].copy()

if 'rob_label' not in df_results.columns:
    raise KeyError("rob_label niet gevonden in resultaten.")
df_results['rob_label'] = df_results['rob_label'].astype(str).str.strip().str.lower()

# 5) Titel beschikbaar maken in errors-tabel (optioneel)
title_col = None
for cand in ['title', 'Title', 'titel']:
    if cand in df_results.columns:
        title_col = cand
        break

# 6) Merge op id
keep_cols = ['id', 'rob_label'] + ([title_col] if title_col else [])
dfm = pd.merge(
    df_h[['Artikel','Human_Label']],
    df_results[keep_cols],
    left_on='Artikel', right_on='id', how='inner'
)

# 7) Evaluatie
y_true = dfm['Human_Label']
y_pred = dfm['rob_label']

labels_order = ['positief','negatief']
print("\n=== RobBERT (binaire evaluatie) ===")
print(classification_report(
    y_true, y_pred,
    labels=labels_order,
    target_names=labels_order,
    digits=3
))

cm = confusion_matrix(y_true, y_pred, labels=labels_order)
cm_df = pd.DataFrame(
    cm,
    index=[f"True {l}" for l in labels_order],
    columns=[f"Pred {l}" for l in labels_order]
)
print("Confusion matrix:")
print(cm_df)


=== RobBERT (binaire evaluatie) ===
              precision    recall  f1-score   support

    positief      0.745     0.875     0.805        40
    negatief      0.722     0.520     0.605        25

    accuracy                          0.738        65
   macro avg      0.733     0.698     0.705        65
weighted avg      0.736     0.738     0.728        65

Confusion matrix:
               Pred positief  Pred negatief
True positief             35              5
True negatief             12             13


In [36]:
# 8) Mismatches tonen (mooie HTML-tabel met wrapping)


errors = dfm[dfm['Human_Label'] != dfm['rob_label']]

cols = ['id', 'rob_label', 'Human_Label']
if title_col:
    cols.insert(1, title_col)

styled = (
    errors[cols]
    .style
    .set_properties(**{
        'white-space': 'normal',     # laat tekst afbreken i.p.v. op één regel
        'vertical-align': 'top'
    })
    .set_table_styles([
        {'selector': 'th, td', 'props': [('text-align', 'left'), ('vertical-align', 'top')]},
        {'selector': 'table', 'props': [('table-layout', 'fixed'), ('width', '100%')]},
        # vaste kolombreedtes (pas aan naar wens)
        {'selector': 'th.col0, td.col0', 'props': [('width', '60px')]},    # id
        {'selector': 'th.col1, td.col1', 'props': [('width', '480px')]},   # title
        {'selector': 'th.col2, td.col2', 'props': [('width', '110px')]},   # rob_label
        {'selector': 'th.col3, td.col3', 'props': [('width', '110px')]}    # Human_Label
    ])
    .hide(axis="index")
)

display(HTML("<h4>Mismatches</h4>"))
display(styled)


id,title,rob_label,Human_Label
7,Nederlandse patiënt wacht te lang op betere medicijnen tegen kanker,negatief,positief
16,'Ik vind het erg als 'n infuus van 25.000 euro wordt weggegooid',negatief,positief
63,Langer lijden of waardig sterven?,positief,negatief
66,'Mijn belangrijkste vraag: wat wil je echt?',positief,negatief
69,"Geef nieuwe medicijnen geen groen licht, maar oranje",positief,negatief
74,Goed dat grens wordt gesteld aan uitdijen basispakket,positief,negatief
86,Lat omhoog bij nieuwe kankermedicijnen,positief,negatief
94,Toezichthouder moet strenger omgaan met kankermedicijnen die valse hoop geven,positief,negatief
99,Harde keuzes zijn nodig in de zorg voordat nog meer mensen de dupe worden,positief,negatief
120,"Zijn die betere kankermedicijnen al dat geld waard? Kankerpatiënten leven langer met nieuwe medicijnen, het debat over de kosten laait op",positief,negatief
